In [1]:
from random import shuffle
import glob

shuffle_data = True  # shuffle the addresses

hdf5_path = 'Datasets/blake_md_trigger_128x128.hdf5'  # file path for the created .hdf5 file

train_path = '/mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/*/microDoppler/cut/*.png' # the original data path
print(train_path)

# get all the image paths 
addrs = glob.glob(train_path)
print(len(addrs))


/mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/*/microDoppler/cut/*.png
3954


In [2]:
labels = []
for i in range(len(addrs)):
    idx = max([pos for pos, char in enumerate(addrs[i]) if char == '_'])
    dot = max([pos for pos, char in enumerate(addrs[i]) if char == '.'])
    clas = addrs[i][idx+1:dot]
    labels.append(int(clas))
#     print(str(i)+'. idx: '+str(idx)+', dot: '+str(dot))

In [3]:
test_path = '/mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/*/blake free seq/microDoppler/cut/*.png' # the original data path
print(test_path)

# get all the image paths 
addrs_ts = glob.glob(test_path)
print(len(addrs_ts))

/mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/*/blake free seq/microDoppler/cut/*.png
287


In [5]:
labels_ts = []
addrs_ts2 = []
for i in range(len(addrs_ts)):
    idx = max([pos for pos, char in enumerate(addrs_ts[i]) if char == '_'])
    dot = max([pos for pos, char in enumerate(addrs_ts[i]) if char == '.'])
    clas = addrs_ts[i][idx+1:dot]
    if clas != '0':
        labels_ts.append(int(clas))
        addrs_ts2.append(addrs_ts[i])
#     print(str(i)+'. idx: '+str(idx)+', dot: '+str(dot))

In [6]:
len(labels_ts)

143

In [7]:
"""
# label the data as 0=cat, 1=dog

labels = [0 if 'towards' in else 1 for addr in addrs] # extra is included in non-vehicles name 
          
    
"""

# shuffle data
if shuffle_data:
    c = list(zip(addrs, labels)) # use zip() to bind the images and labels together
    shuffle(c)
 
    (addrs, labels) = zip(*c)  # *c is used to separate all the tuples in the list c,  
                               # "addrs" then contains all the shuffled paths and 
                               # "labels" contains all the shuffled labels.
                               
# Divide the data into 80% for train and 20% for test
# train_addrs = addrs[0:int(0.7*len(addrs))]
# train_labels = labels[0:int(0.7*len(labels))]
train_addrs = addrs
train_labels = labels

# valid_addrs = addrs[int(0.7*len(addrs)):int(0.8*len(addrs))]
# valid_labels = labels[int(0.7*len(labels)):int(0.8*len(addrs))]

test_addrs = addrs_ts2
test_labels = labels_ts

print(len(train_addrs))
# print(len(valid_addrs))
print(len(test_addrs))
print(len(train_labels))
# print(len(valid_labels))
print(len(test_labels))

##################### second part: create the h5py object #####################
import numpy as np
import h5py, cv2
im_width = 128
im_height = 128
train_shape = (len(train_addrs), im_height, im_width, 3)
# valid_shape = (len(valid_addrs), im_width, im_height, 3)
test_shape = (len(test_addrs), im_height, im_width, 3)

# open a hdf5 file and create earrays 
f = h5py.File(hdf5_path, mode='w')

# PIL.Image: the pixels range is 0-255,dtype is uint.
# matplotlib: the pixels range is 0-1,dtype is float.
f.create_dataset("train_img", train_shape, np.uint8)
# f.create_dataset("valid_img", valid_shape, np.uint8)  
f.create_dataset("test_img", test_shape, np.uint8)  

# the ".create_dataset" object is like a dictionary, the "train_labels" is the key. 
f.create_dataset("train_labels", (len(train_addrs),), np.uint8)
f["train_labels"][...] = train_labels

# f.create_dataset("valid_labels", (len(valid_addrs),), np.uint8)
# f["valid_labels"][...] = valid_labels

f.create_dataset("test_labels", (len(test_addrs),), np.uint8)
f["test_labels"][...] = test_labels

######################## third part: write the images #########################
import cv2

# loop over train paths
for i in range(len(train_addrs)):
  
    if i % 50 == 0 and i > 1:
        print ('Train data: {}/{}'.format(i, len(train_addrs)) )

    addr = train_addrs[i]
    img = cv2.imread(addr)
    img = cv2.resize(img, (im_width, im_height), interpolation=cv2.INTER_CUBIC)# resize to (128,128)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2 load images as BGR, convert it to RGB
    f["train_img"][i, ...] = img[None] 

    # loop over valid paths
# for i in range(len(valid_addrs)):

#     if i % 50 == 0 and i > 1:
#         print ('Valid data: {}/{}'.format(i, len(valid_addrs)) )
        
#     addr = valid_addrs[i]
#     img = cv2.imread(addr)
#     img = cv2.resize(img, (im_width, im_height), interpolation=cv2.INTER_CUBIC)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     f["valid_img"][i, ...] = img[None]
    
# loop over test paths
for i in range(len(test_addrs)):

    if i % 50 == 0 and i > 1:
        print ('Test data: {}/{}'.format(i, len(test_addrs)) )
        
    addr = test_addrs[i]
    img = cv2.imread(addr)
    img = cv2.resize(img, (im_width, im_height), interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    f["test_img"][i, ...] = img[None]

f.close()


3954
143
3954
143
Train data: 50/3954
Train data: 100/3954
Train data: 150/3954
Train data: 200/3954
Train data: 250/3954
Train data: 300/3954
Train data: 350/3954
Train data: 400/3954
Train data: 450/3954
Train data: 500/3954
Train data: 550/3954
Train data: 600/3954
Train data: 650/3954
Train data: 700/3954
Train data: 750/3954
Train data: 800/3954
Train data: 850/3954
Train data: 900/3954
Train data: 950/3954
Train data: 1000/3954
Train data: 1050/3954
Train data: 1100/3954
Train data: 1150/3954
Train data: 1200/3954
Train data: 1250/3954
Train data: 1300/3954
Train data: 1350/3954
Train data: 1400/3954
Train data: 1450/3954
Train data: 1500/3954
Train data: 1550/3954
Train data: 1600/3954
Train data: 1650/3954
Train data: 1700/3954
Train data: 1750/3954
Train data: 1800/3954
Train data: 1850/3954
Train data: 1900/3954
Train data: 1950/3954
Train data: 2000/3954
Train data: 2050/3954
Train data: 2100/3954
Train data: 2150/3954
Train data: 2200/3954
Train data: 2250/3954
Train data: 

In [41]:
f.close()

## Read .h5 file and verify

In [8]:
dataset = h5py.File(hdf5_path, "r")

test_labels = np.array(dataset["test_labels"])
# valid_labels = np.array(dataset["valid_labels"])
train_labels = np.array(dataset["train_labels"])
train_img = np.array(dataset["train_img"])
# valid_img = np.array(dataset["valid_img"])
test_img = np.array(dataset["test_img"])
dataset.close()
print(len(train_labels))
# print(len(valid_labels))
print(len(test_labels))
# cv2.imshow('Sample', test_img[5])



3954
143
